In [3]:
## Import the necessary packages
from bs4 import BeautifulSoup
import urllib
import re
import pandas as pd
import requests
import bs4
from urllib.request import urlopen
import time
import seaborn as sns

In [4]:
url ="https://www.indeed.com/cmp?q=data+scientist&l=los+angeles&from=discovery"
page = requests.get(url)

main_soup = bs4.BeautifulSoup(page.text,"html.parser")
body = main_soup.html

In [5]:
company_details = main_soup.find_all("div", class_="clearfix cmp-CompanyWidget")

In [6]:
#creating a list of dictionaries that stores the name of the company as well as the link to the review page

info = []
main_url = "https://www.indeed.com"
for c in company_details:
    name_reviews = {}
    name = c.find_all("a", class_="cmp-CompanyWidget-name")[0].text
    review_all = c.find_all("div", class_="cmp-CompanyWidget-links")
    review = review_all[0].a["href"]
    
    name_reviews["name"] = name
    name_reviews["review_link"]= main_url + review
    
    info.append(name_reviews)

In [7]:
info

[{'name': 'Veritone',
  'review_link': 'https://www.indeed.com/cmp/Veritone/reviews'},
 {'name': 'Hulu', 'review_link': 'https://www.indeed.com/cmp/Hulu/reviews'},
 {'name': 'Deloitte',
  'review_link': 'https://www.indeed.com/cmp/Deloitte/reviews'},
 {'name': 'City National Bank',
  'review_link': 'https://www.indeed.com/cmp/City-National-Bank/reviews'},
 {'name': 'University of California Los Angeles',
  'review_link': 'https://www.indeed.com/cmp/University-of-California-Los-Angeles/reviews'},
 {'name': 'ZS Associates',
  'review_link': 'https://www.indeed.com/cmp/Zs-Associates/reviews'},
 {'name': 'Snap Inc.',
  'review_link': 'https://www.indeed.com/cmp/Snap-Inc./reviews'},
 {'name': 'UCLA Health',
  'review_link': 'https://www.indeed.com/cmp/Ucla-Health/reviews'},
 {'name': 'University of Southern California',
  'review_link': 'https://www.indeed.com/cmp/University-of-Southern-California/reviews'},
 {'name': 'Next Trucking',
  'review_link': 'https://www.indeed.com/cmp/Next-Trucki

In [8]:
#this function gets all the relevant information from each page
def get_data(name, container_list):
    title_all = []
    rating_all =[]
    review_all = []
    pro_all = []
    con_all = []
    date = []
    title_of_reviewer = []
    
    for c in container_list:

        title = c.find_all("div", class_="cmp-review-title")[0].span.text
        rating = c.find_all("div", class_="cmp-ratingNumber")[0].text
        review = c.find_all("span", class_="cmp-review-text")[0].text
        reviewer_title = c.find_all("span",class_ = "cmp-reviewer")[0].text
        review_date = c.find_all("span",class_ = "cmp-review-date-created")[0].text
        


        #for some reviews the pros and cons are not available - so giving them None value
        pro = None
        con = None

        pro_available = c.find_all("div", class_="cmp-review-pro-text")
        con_available = c.find_all("div", class_="cmp-review-con-text") 

        if pro_available: #i.e if it is a non-empty list
            pro = c.find_all("div", class_="cmp-review-pro-text")[0].text
        if con_available: #i.e if it is a non-empty list
            con = c.find_all("div", class_="cmp-review-con-text")[0].text
            
        title_all.append(title)
        rating_all.append(rating)
        review_all.append(review)
        title_of_reviewer.append(reviewer_title)
        date.append(review_date)
        pro_all.append(pro)
        con_all.append(con)
        
    job = {'company':name, 'title': title_all,'rating': rating_all,'review': review_all,"title_of_reviewer":title_of_reviewer,"date":date,'pros': pro_all,'cons':con_all}
    df = pd.DataFrame.from_dict(job)
    
    
    return df

In [10]:
#write a function that takes a url and gives u containers


def container_collection(url):
    #the issue is at times when a request is placed, the server returns nothing. 
    #therefore, need to write a code that takes care of that issue and hits a website again to get data
    
    counter = 0
    retries = 0
    retries_next = 0
    
    main_url="https://www.indeed.com"
    
    
    while retries <= 4:
        #if the server sends back data need to get container from it
        job = requests.get(url)
        soup = bs4.BeautifulSoup(job.text, "lxml")
        container = soup.find_all("div", class_="cmp-review-container")
        
        #all good - found container
        if len(container) > 0:
            break 
        #try again
        else:
            retries += 1
    #tried again but did not get container? return an empty list : this way the code wont break
    if not container:
        return []
    
    #since we need total 50 reviews
    while len(container) < 50:
        
        #find the link of the next page on the main page
        next_link = soup.find_all("div", class_="cmp-Pagination")
        
        if next_link:   
            #find the url for the next page and fetch data from it
            next_url=next_link[0].a['href']
            new_url =  main_url + next_url
            url = new_url
            
            # now try to get some containers from next link 4 times
            while retries_next <= 4:
                                
                job = requests.get(url)
                soup = bs4.BeautifulSoup(job.text, "lxml")
                container_next = soup.find_all("div", class_="cmp-review-container")
                
                # if containers were found
                if len(container_next) > 0:
                    # great, reset retries_next and get out of the retry-loop
                    retries_next = 0
                    break 

                # if no containers were found    
                #try again by increasing retries_next and continuing to the next retry loop
                else:
                    retries_next += 1
                    continue
            
            # if at any time outside of retries
            # you find a container
            # add it to the big list
            if container_next:
                container += container_next
                
            # and then continue to the next next link
            continue
        
        #if next link is not available
        else:
            # just return the big list
            return container[0:50]
         

    #for all the companies, that have more than 50 reviews, picking only 50
    return container[0:50]

In [46]:

#calling all functions here
main_df = pd.DataFrame()
for i in info:
    containers = container_collection(i['review_link'])
    df = get_data(i['name'], containers)
    main_df = main_df.append(df,ignore_index=True)

### SOLUTION DATASET

In [47]:

main_df

,company,title,rating,review,title_of_reviewer,date,pros,cons
0,Veritone,"Has potential, but job stability is always in ...",3.0,Too many parts moving at the same time with un...,Hopefully not randomly laid-off soon,"January 7, 2018",Cool new industry,Random unexplained lay-offs
1,Veritone,"Great people, but planning leaves something to...",3.0,- Relaxed work hours where most managers don't...,Software Engineer,"June 24, 2019",None,None
2,Veritone,Good company with creative atomsphere,5.0,A company offers good atmosphere at creative t...,Data Scientist Intern,"January 2, 2019",None,None
3,Veritone,Start up company with amazing AI promise,3.0,Great AI resources coming out of this soon to ...,Director New Business Development,"October 2, 2017",Cutting edge company,Start ups usual challenges
4,Hulu,It's okay,3.0,I loved working here in the beginning. Once th...,Viewer Experience Advocate,"October 25, 2019",None,None
5,Hulu,Was a model of a new kind of company but is no...,3.0,In the beginning Hulu was a passion project ca...,Marketing/Sales Ops,"October 19, 2019","really work hard to keep things fun, holiday p...","managers from old world media, turned corporat..."
6,Hulu,Amazing Place to Work!,5.0,Amazing place to work. Fast paced and interest...,"Data Science Manager, Advanced Analytics","October 4, 2019",None,None
7,Hulu,"Environment filled with perks and fun, but not...",4.0,Working at Hulu feels like living on a college...,Digital Supply Chain Operations Specialist,"October 1, 2019","A lot of free food, good insurance",A lot of work with little advancement and comp...
8,Hulu,Can’t have a life.,5.0,"Don’t have kids, or go to school with this com...",VXA,"September 28, 2019",None,None
9,Hulu,Innovative and stimulating place to work.,5.0,Hulu is an incredible place to be employed. A...,Broadcast Operations Technician,"September 25, 2019",Benefits,Salary is less than other places of equivalent...


In [13]:
main_df.to_csv("all_reviews.csv")